In [22]:
import pyodbc
import pandas as pd
import shutil
import re

def copy_database(original_path, temp_path):
    shutil.copyfile(original_path, temp_path)

def export_access_to_dataframes(database_path):
    # Connection string for Access database
    conn_str = (
        r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
        r'DBQ=' + database_path + ';'
    )

    # Establish a connection to the Access database
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()

    # Get a list of all tables in the database
    tables = [row.table_name for row in cursor.tables(tableType='TABLE')]

    # Loop through the tables and load each into a DataFrame
    for table in tables:
        query = f'SELECT * FROM [{table}]'
        df = pd.read_sql(query, conn)
        globals()[f'tbl_{table}'] = df  # Create a global variable with the table name

    # Close the connection
    conn.close()

def update_access_table(database_path, table_name, df):
    # Connection string for Access database
    conn_str = (
        r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
        r'DBQ=' + database_path + ';'
    )

    # Establish a connection to the Access database
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()

    # Clear the existing table
    cursor.execute(f'DELETE FROM [{table_name}]')
    conn.commit()

    # Insert the updated data
    for index, row in df.iterrows():
        columns = ', '.join(row.index)
        placeholders = ', '.join(['?' for _ in row])
        values = tuple(row)
        sql = f'INSERT INTO [{table_name}] ({columns}) VALUES ({placeholders})'
        cursor.execute(sql, values)
    conn.commit()

    # Close the connection
    conn.close()

# Paths
original_database_path = 'E:\\digidure\\CLERUS_v1_24072024.accdb'
temp_database_path = 'E:\\digidure\\CLERUS_temp.accdb'

# Copy the database
copy_database(original_database_path, temp_database_path)

# Export data from the copied database
export_access_to_dataframes(temp_database_path)

C:\Users\Maurice de Kleijn\AppData\Local\Temp\ipykernel_22940\3741754444.py:26: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
C:\Users\Maurice de Kleijn\AppData\Local\Temp\ipykernel_22940\3741754444.py:26: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
C:\Users\Maurice de Kleijn\AppData\Local\Temp\ipykernel_22940\3741754444.py:26: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
C:\Users\Maurice de Kleijn\AppData\Local\Temp\ipykernel_22940

In [23]:
# Panda settings for showing data (this is foremost done to more easily explore the data while processing it)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [24]:
# All the individuals that could be matched with DRC are already curated and processed. The next step is to extract individuals from DM that are not in DRC. 
# When a link between DM and DRC was found, a value for clerus_id or new_clerus_id was produced. Thus to isolate those that have not been matched we create 
# a subselection existing of rows where these fields are empty.
filtered_only_DM = tbl_999_Dm_all_drc_match[tbl_999_Dm_all_drc_match['clerus_id'].isna() & tbl_999_Dm_all_drc_match['new_clerus_id'].isna()]


In [25]:
# Since we want to link the individuals to clerus we want to use the individual_id lateron as a new clerus_id. To avoid overlap in numbers we added 9000000 to every id. The individual ids have been created through 2_1_check_links_DRC_DM.ipynb
filtered_only_DM["individual_id"] = filtered_only_DM["individual_id"] + 9000000

C:\Users\Maurice de Kleijn\AppData\Local\Temp\ipykernel_22940\3344831569.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_only_DM["individual_id"] = filtered_only_DM["individual_id"] + 9000000


In [26]:
filtered_only_DM.head(15)

,pid,ind_id,provincie,classis,gemeente,wijk,predikant,Herkomst,dag intrede,maand intrede,jaar intrede,vertrek naar of vanwege,dag vertrek,maand vertrek,jaar vertrek,Bijzonderheden,individual_id,clerus_id,new_clerus_id
0,35651,1,ZH,None,Oudewater,None,(Hekendorp) Brunner; L.,hulppredikant Oirschot,None,None,1867,Lobith,None,None,1868,None,9013955,NaN,NaN
1,45546,2,GE,None,Velp,2. Oude Jan,(Hilten van-)Matthijsen;mevrouw L.,kandidaat,2,maart,1986,Apeldoorn,23,oktober,1994,None,9013956,NaN,NaN
2,7179,3,NB,None,Breda,None,Aa; dr Pieter Jan Baptist Karel Simon van der,Winterswijk,26,februari,1893,emeritaat,1,mei,1904,None,9013962,NaN,NaN
3,21505,3,GR,None,Hornhuizen,None,Aa; dr. P.J.B.K. Simon van der,kandidaat,None,None,1862,Goutum,None,None,1866,None,9013959,NaN,NaN
4,36887,3,NH,None,Purmerend,None,Aa; dr. P.J.B.K. Simon van der,Goutum,None,None,1886,Winterswijk,None,None,1892,None,9013960,NaN,NaN
5,50146,3,GE,None,Winterswijk,None,Aa; dr. P.J.B.K. Simon van der,Purmerend,None,None,1892,Breda,None,None,1893,None,9013960,NaN,NaN
6,15179,3,FR,None,Goutum,None,Aa; dr. P.J.B.K. van der,Kloosterburen,None,None,1866,Purmerend,None,None,1886,None,9013961,NaN,NaN
7,7204,3,NB,None,Breda,4. Wijk Noord,Aa; dr. P.J.B.K.S. van der,Winterswijk,26,februari,1893,emeritaat,1,mei,1904,None,9013958,NaN,NaN
8,19779,4,GE,None,Herwijnen,None,Aa; W.P. van der,kandidaat,24,december,1989,None,None,None,None,None,9013957,NaN,NaN
9,35941,5,OV,None,Overdinkel,None,Aaij; J.,Hoogkarspel-De Drieslag,25,september,1983,emeritaat,1,september,1986,None,9013963,NaN,NaN


In [ ]:
#From this dataset we will be extracting all kind of information to align with the Clerus database structure.

In [50]:
#First we extract information for every individual which we use to fill in the table 01_clerus_bio. These are the names of the individual, the year of death and remarks (Bijzonderheden).
# since we are not interested in all the other fields we make a subselection

DM_tbl_bio_1 = filtered_only_DM[['pid', 'predikant', "vertrek naar of vanwege", "jaar vertrek", "Bijzonderheden", "individual_id"]]

In [51]:
# Extract Year of Death 
# After analysing the data in DM it appears that information about someone's year of death can be found in the field "Bijzonderheden", but also in the field "vertrek naar of vanwege". 
# To extract this information and to save it in a new field is done in two steps.

# first we extract year from Bijzonderheden where it contains the string "overl." or "overleden"
def extract_year(bijzonderheden):
    if isinstance(bijzonderheden, str):
        match = re.search(r'(overleden|overl\.)\s*(\d{4})', bijzonderheden, re.IGNORECASE)
        if match:
            return int(match.group(2))
    return None

DM_tbl_bio_1['year_death'] = DM_tbl_bio_1['Bijzonderheden'].apply(extract_year)

# Second we update 'year_death' where it is None, using 'jaar_vertrek' if 'vertrek_naar_of_vanwege' contains 'overleden' or 'overl.'
DM_tbl_bio_1['year_death'] = DM_tbl_bio_1.apply(
    lambda row: row['jaar vertrek'] 
    if pd.isnull(row['year_death']) and 
       isinstance(row['vertrek naar of vanwege'], str) and 
       ('overleden' in row['vertrek naar of vanwege'].lower() or 'overl.' in row['vertrek naar of vanwege'].lower())
    else row['year_death'], 
    axis=1
)

C:\Users\Maurice de Kleijn\AppData\Local\Temp\ipykernel_22940\851195143.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DM_tbl_bio_1['year_death'] = DM_tbl_bio_1['Bijzonderheden'].apply(extract_year)
C:\Users\Maurice de Kleijn\AppData\Local\Temp\ipykernel_22940\851195143.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DM_tbl_bio_1['year_death'] = DM_tbl_bio_1.apply(


In [52]:
# Next we need to make another subselection of the data to get only the name, clerus_id, year of death and remarks. 
DM_tbl_bio_2 = DM_tbl_bio_1[['predikant', "Bijzonderheden", "individual_id", "year_death"]]

In [68]:
DM_tbl_bio_2['year_death'] = DM_tbl_bio_2['year_death'].fillna(0).astype(int)
print(DM_tbl_bio_2.dtypes)

predikant         object
Bijzonderheden    object
individual_id      int64
year_death         int32
dtype: object


C:\Users\Maurice de Kleijn\AppData\Local\Temp\ipykernel_22940\936621422.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DM_tbl_bio_2['year_death'] = DM_tbl_bio_2['year_death'].fillna(0).astype(int)


In [69]:
# Now we remove all duplicates that remain based on the inidivual_id and prioritize on whether "year_death" has a value. 

DM_tbl_bio_2 = DM_tbl_bio_2.sort_values(by='year_death', ascending=False, na_position='last')



In [70]:
DM_tbl_bio_2.describe()

,individual_id,year_death
count,2.946300e+04,29463.000000
mean,9.022581e+06,168.277399
std,5.430808e+03,539.533276
min,9.000025e+06,0.000000
25%,9.018184e+06,0.000000
50%,9.022633e+06,0.000000
75%,9.027134e+06,0.000000
max,9.031653e+06,2005.000000


In [71]:
# Drop duplicates based on 'individual_id', keeping the first occurrence
DM_tbl_bio_unique = DM_tbl_bio_2.drop_duplicates(subset='individual_id', keep='first')

In [72]:
DM_tbl_bio_unique.describe()

,individual_id,year_death
count,1.746000e+04,17460.000000
mean,9.022504e+06,279.286025
std,5.558771e+03,672.406587
min,9.000025e+06,0.000000
25%,9.018143e+06,0.000000
50%,9.022632e+06,0.000000
75%,9.027147e+06,0.000000
max,9.031653e+06,2005.000000


In [11]:
# Update the table in the copied database
update_access_table(temp_database_path, '12_clerus_role', df_joined)